In [1]:
from CyRK import pysolve_ivp, WrapCySolverResult, __version__

print(__version__)

import Cython

0.11.6a0.dev0


In [2]:
%load_ext cython

In [ ]:
%%cython -a -f
# distutils: language = c++
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

import numpy as np
cimport numpy as np
np.import_array()

from libcpp.vector cimport vector

from CyRK cimport cysolve_ivp, CySolveOutput, CySolverResult, DiffeqFuncType
from CyRK.cy.cysolver_test cimport lotkavolterra_diffeq


cdef DiffeqFuncType diffeq = lotkavolterra_diffeq


cdef CySolveOutput test_1():

    cdef double[2] t_span = [0., 10.]
    cdef double* t_span_ptr = &t_span[0]

    cdef double[2] y0 = [10., 5.]
    cdef double* y0_ptr = &y0[0]

    cdef size_t num_y = 2

    cdef vector[double] args = vector[double]()
    args.resize(3)
    args[0] = 1.0
    args[1] = 1.0
    args[2] = 9.81
    cdef double* args_ptr = args.data()

    cdef CySolveOutput result = cysolve_ivp(
        diffeq,
        t_span_ptr,
        y0_ptr,
        num_y,
        1,
        1.0e-5,
        1.0e-6,
        args_ptr,
        0,
        1_000_000,
        2_000,
        True,
        NULL,
        0,
        NULL,
        NULL,
        NULL,
        10_000,
        0.0,
        100
        )
    return result

from libc.stdlib cimport malloc, free, realloc

cdef CySolveOutput test_2():

    cdef double[2] t_span = [0., 10.]
    cdef double* t_span_ptr = &t_span[0]

    cdef double[2] y0 = [10., 5.]
    cdef double* y0_ptr = &y0[0]

    cdef size_t num_y = 2

    cdef double* args_ptr = <double*>malloc(sizeof(double)*3)
    args_ptr[0] = 1.0
    args_ptr[1] = 1.0
    args_ptr[2] = 9.81

    cdef CySolveOutput result = cysolve_ivp(
        diffeq,
        t_span_ptr,
        y0_ptr,
        num_y,
        1,
        1.0e-5,
        1.0e-6,
        args_ptr,
        0,
        1_000_000,
        2_000,
        True,
        NULL,
        0,
        NULL,
        NULL,
        NULL,
        10_000,
        0.0,
        100
        )
    realloc(args_ptr, sizeof(double)*3000)
    cdef size_t i 
    for i in range(3000):
        args_ptr[i] = -1.0
    free(args_ptr)
    return result

cdef CySolveOutput res_shptr
cdef CySolverResult* res

from libc.stdio cimport printf

cdef double[2] y_interp
cdef double* y_interp_ptr = &y_interp[0]
cdef size_t i

printf("\nTest 1\n")
for i in range(10):
    printf("\tSubTest Num = %d\n", i)
    res_shptr = test_1()
    res = res_shptr.get()
    printf("Test 1 Success = %d\n", res.success)
    
    y_interp_ptr[0] = 0.0
    y_interp_ptr[1] = 0.0
    
    printf("Test 1; calling\n")
    res.call(4.38, y_interp)
    printf("Test 1; Call Finished. y0 = %e; y1 = %e\n", y_interp[0], y_interp[1])

printf("\nTest 2\n")
for i in range(10):
    printf("\tSubTest Num = %d\n", i)
    res_shptr = test_2()
    res = res_shptr.get()
    printf("Test 2 Success = %d\n", res.success)
    
    y_interp_ptr[0] = 0.0
    y_interp_ptr[1] = 0.0
    
    printf("Test 2; calling\n")
    res.call(4.38, y_interp)
    printf("Test 2; Call Finished. y0 = %e; y1 = %e\n", y_interp[0], y_interp[1])

In [10]:
def test_1():
    result = \
        pysolve_ivp(diffeq_args_extra, time_span, initial_conds,
                    method="RK45",
                    args=args, rtol=rtol, atol=atol,
                    dense_output=True,
                    num_extra=2, first_step=0.0, max_step=1000.0,
                    pass_dy_as_arg=True)
    return result
res_1 = test_1()
print(res_1.success, res_1.message)
print(res_1(5.2))

True Integration completed without issue.
[[  7.03679027]
 [124.35084124]
 [ -0.24350841]
 [ -0.85926419]]


In [372]:
def test_2():
    args_2 = (0.01, 0.02)
    
    result = \
        pysolve_ivp(diffeq_args_extra, time_span, initial_conds,
                    method="RK45",
                    args=args_2, rtol=rtol, atol=atol,
                    dense_output=True,
                    num_extra=2, first_step=0.0, max_step=1000.0,
                    pass_dy_as_arg=True)
    del args_2
    return result
res_2 = test_2()
print(res_2.success, res_2.message)
print(res_2(5.2))

True Integration completed without issue.
[[  7.03679027]
 [124.35084124]
 [ -0.24350841]
 [ -0.85926419]]
